In [ ]:
# CELL 1 — Install Dependencies & Config

!pip install gdown ffmpeg-python jiwer transformers
!pip install git+https://github.com/openai/whisper.git
!apt-get install ffmpeg -y

import os, json, math, datetime
from pathlib import Path
import numpy as np
import ffmpeg
import gdown
import whisper
import torch
from jiwer import wer

print("Torch device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# CONFIG — list video Drive IDs
DRIVE_IDS = [
    "1pVqgMe9g3_3AVxpf2odq3kWfJSOGyFkc",
    "1bt2Sx5FVdr4ONV_Bz-0vF6ym2bar0xF5",
    "1ye5yChiUE4lY0KL5Xotc-dJWI5W-WwYe",
    "1DtY3Pnxv5rFSbENYHxOyW-8TlqZLECq0",
    "1nykD4nX2A94DrSwf9PdZi-4abXmxZcKP"
]

QUESTIONS = {
    1: "Can you share any specific challenges you faced while working on certification and how you overcame them?",
    2: "Can you describe your experience with transfer learning in TensorFlow? How did it benefit your projects?",
    3: "Describe a complex TensorFlow model you have built and the steps you took to ensure its accuracy and efficiency.",
    4: "Explain how to implement dropout in a TensorFlow model and the effect it has on training.",
    5: "Describe the process of building a convolutional neural network (CNN) using TensorFlow for image classification."
}

OUT_DIR = "capstone_output"
VID_DIR = os.path.join(OUT_DIR, "videos")
AUD_DIR = os.path.join(OUT_DIR, "audios")
os.makedirs(VID_DIR, exist_ok=True)
os.makedirs(AUD_DIR, exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xxzf403l
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xxzf403l
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=dfe233d315de2a269c1f633150f2a49274c47e8c7e1ad83caa937c833a974f36
  Stored in directory: /tmp/pip-ephem-wheel-cache-fdafsqbx/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.

In [ ]:
# CELL 2 — Download raw video files

def download_drive(id_, dest):
    url = f"https://drive.google.com/uc?id={id_}"
    print("Downloading:", id_)
    gdown.download(url, dest, quiet=False)

video_paths = []
for i, fid in enumerate(DRIVE_IDS, start=1):
    out = os.path.join(VID_DIR, f"interview_question_{i}.mp4")
    download_drive(fid, out)
    video_paths.append(out)

print("Downloaded files:", video_paths)

Downloading: 1pVqgMe9g3_3AVxpf2odq3kWfJSOGyFkc


Downloading...
From: https://drive.google.com/uc?id=1pVqgMe9g3_3AVxpf2odq3kWfJSOGyFkc
To: /content/capstone_output/videos/interview_question_1.mp4
100%|██████████| 18.0M/18.0M [00:00<00:00, 38.5MB/s]


Downloading: 1bt2Sx5FVdr4ONV_Bz-0vF6ym2bar0xF5


Downloading...
From: https://drive.google.com/uc?id=1bt2Sx5FVdr4ONV_Bz-0vF6ym2bar0xF5
To: /content/capstone_output/videos/interview_question_2.mp4
100%|██████████| 22.0M/22.0M [00:00<00:00, 63.8MB/s]


Downloading: 1ye5yChiUE4lY0KL5Xotc-dJWI5W-WwYe


...

Downloading: 1DtY3Pnxv5rFSbENYHxOyW-8TlqZLECq0


Downloading...
From: https://drive.google.com/uc?id=1DtY3Pnxv5rFSbENYHxOyW-8TlqZLECq0
To: /content/capstone_output/videos/interview_question_4.mp4
100%|██████████| 23.1M/23.1M [00:00<00:00, 37.8MB/s]


Downloading: 1nykD4nX2A94DrSwf9PdZi-4abXmxZcKP


Downloading...
From: https://drive.google.com/uc?id=1nykD4nX2A94DrSwf9PdZi-4abXmxZcKP
To: /content/capstone_output/videos/interview_question_5.mp4
100%|██████████| 23.0M/23.0M [00:00<00:00, 51.9MB/s]

Downloaded files: ['capstone_output/videos/interview_question_1.mp4', 'capstone_output/videos/interview_question_2.mp4', 'capstone_output/videos/interview_question_3.mp4', 'capstone_output/videos/interview_question_4.mp4', 'capstone_output/videos/interview_question_5.mp4']


In [ ]:
# CELL 3 — Convert MP4 files to WAV audio

def convert_to_wav(mp4_path, wav_path):
    (
        ffmpeg
        .input(mp4_path)
        .output(wav_path, ac=1, ar="16000")
        .overwrite_output()
        .run(quiet=True)
    )

wav_paths = []
for mp4 in video_paths:
    base = Path(mp4).stem
    wav = os.path.join(AUD_DIR, base + ".wav")
    print(f"Converting {mp4} -> {wav}")
    convert_to_wav(mp4, wav)
    wav_paths.append(wav)

print("All conversions done.")

Converting capstone_output/videos/interview_question_1.mp4 -> capstone_output/audios/interview_question_1.wav
Converting capstone_output/videos/interview_question_2.mp4 -> capstone_output/audios/interview_question_2.wav
Converting capstone_output/videos/interview_question_3.mp4 -> capstone_output/audios/interview_question_3.wav
Converting capstone_output/videos/interview_question_4.mp4 -> capstone_output/audios/interview_question_4.wav
Converting capstone_output/videos/interview_question_5.mp4 -> capstone_output/audios/interview_question_5.wav
All conversions done.


In [ ]:
# CELL 4 — Load Whisper Model

MODEL_NAME = "large"   # medium jika RAM kecil
print("Loading model:", MODEL_NAME)
model = whisper.load_model(MODEL_NAME)
print("Model loaded.")

Loading model: large


100%|█████████████████████████████████████| 2.88G/2.88G [00:43<00:00, 71.2MiB/s]


Model loaded.


In [ ]:
# CELL 5 — Compute confidence scores

def seg_confidence_from_avglogprob(avg_logprob):
    try:
        p = math.exp(avg_logprob)
    except:
        p = 0.0
    return float(max(0.0, min(1.0, p)))

def overall_confidence_from_segments(segments):
    if not segments:
        return 0.0
    weights = [(s['end'] - s['start']) for s in segments]
    confs = [seg_confidence_from_avglogprob(s.get('avg_logprob', -10)) for s in segments]
    total_w = sum(weights)
    return float(sum(w*c for w,c in zip(weights,confs)) / total_w)

In [ ]:
#Ground Truth
references = {
    "interview_question_1": """share any specific challenges you faced while working on certification and how you overcome them ah, ok actually for the challenges, there are some challenges when I took the certification especially for the project submission that I already working with it the first one is actually to meet the specific accuracy or violation laws for the evaluation matrix and yeah actually that's just need to take some trial and error with the different architecture for example like we can try to add more layer, more neurons, changes the neurons or even I also apply the dropout layer so it really helps with the validation loss to become more lower and I think that's one of the biggest challenges that I have while working on this certification.""",

    "interview_question_2": """Can you describe your experience with transfer learning and time short flow? How do you benefit from the projects? About transfer learning is actually we use existing train model from time short flow for example like VGC16, VGC19 especially for some cases that we need to use deep learning using Keras applications for example like image classification we can use transfer learning models which is that's already train model with exceptionally high accuracy, high performance, even though it's trained with different datasets but it really helps to improve our model performance, model accuracy, model loss and you know for example like mobile net, VGG19, VGG16, efficient net, it will help to improve our our models comparing to the one if you use a traditional CNN model yeah CNN model with the convolutional 2d yeah max pooling and yeah it it's it's quite good actually to use transfer learning it really helps with our model performance to improve our model performance.""",

    "interview_question_3": """wait what is this? describe a complex TensorFlow model you have built and the steps you took to ensure its accuracy and efficiency hmmm, a complex TensorFlow model you have built and steps you took to ensure its accuracy okay I will take one of my previous project that I use yeah I also use Keras or TensorFlow model it's it is about a celiac disease prediction yeah yeah this is also I use the research project for my undergraduate thesis yeah for my script C and I use this model it's quite challenging even though it's achieved high accuracy with some dense layer, with some drawout layer and trial and error also with the callback function with the neurons but the problem is the dataset is not balanced so it has the imbalanced class datasets and the approach that I use is to just to use the technique called smooth and synthetic oversampling technique with edited nearest neighbor which is basically it's just oversampling and undersampling the datasets it helps with the accuracy.""",

    "interview_question_4": """explain how you implement dropout in test on flow model and test on training previously i also have implement the dropout layer also in the project submission within this certifications and we can just add the dropout layer for example if I'm not mistaken it's I have used this dropout layer in the the one that the case is image classifications yeah a German traffic something if I'm not wrong I have used this dropout layer in in in the not in the last in the middle of the layer so there's a flattened layer right not flatten the convolutional layer and then the flattened layer and I use that dropout layer which is I use with the rate of 0.2 or 0.5 if I'm not wrong and then the dense layer and the last the output layer right the effect is it will really helps to improve our accuracy and lower our validation loss by turning off some of the previous layer, for example like we have dense layer 64 and the next layer we implement the dropout layer with the rate of 0.5 and it will turn off randomly each epoch of the previous dense layer""",

    "interview_question_5": """Describe the process of building or configuration of your image application okay, the CNN one, right? so, at the first time, of course, we need to make sure there are split, the image folder is split for each class and then we can use Keras preprocessing image dataset from directory to split the training and the validation dataset of course we can use another set which is the test dataset but yeah okay the next one we can we can just maybe we we need to implement also the image augmentation yeah data data image data augmentation to to make our data set more veritive right for example like we can rotate we can zoom it we can crop it yeah and yeah the last thing yeah of course we we can build our chain model with the convolutional 2D specify the filters the kernel size the LR activation of course the input shape for the first layer and then we can apply the max pooling 2D yeah and the next layer we can just use use conflux 2d, mix pooling and whatever it is and after that we apply the flatten layer and dropout layer if you want and the last thing don't forget to use the dance layer right for the output."""
}

In [ ]:
import re
import random

# Normalization helpers
def normalize_text(text: str) -> str:
    return re.sub(r"[^a-z0-9\s]", " ", text.lower()).strip()

def contains_any(text: str, keywords: list) -> bool:
    return any(kw in text for kw in keywords)

# Helper untuk memilih 1 reason saja
def pick_one_reason(reason_text: str) -> str:
    lines = [line.strip() for line in reason_text.split("\n") if line.strip()]
    return random.choice(lines) if lines else reason_text


RUBRIC = {
    1: {
        "levels": {
            4: (
                "Comprehensive and Clear Response.\n"
                "Provides a detailed description of specific challenges encountered during the certification.\n"
                "Offers clear explanations of how each challenge was overcome.\n"
                "Demonstrates strong understanding of technical aspects and problem-solving skills.\n"
                "Response is very clear, well-organized, and reflects the learning process."
            ),
            3: (
                "Specific Challenge with Basic Solution.\n"
                "Describes at least one specific challenge related to building machine learning models with TensorFlow.\n"
                "Provides a basic explanation of how the challenge was overcome.\n"
                "Explanation may be brief or lack depth.\n"
                "Shows some understanding but lacks detailed insight."
            ),
            2: "General Challenge Mentioned without Details.",
            1: "Minimal or Vague Response.",
            0: "Unanswered"
        },
        "keywords": {
            4: ["challenge","overcome","solution","error","architecture","certification"],
            3: ["challenge","problem","difficult"],
            2: ["problem"],
            1: []
        }
    },

    2: {
        "levels": {
            4: (
                "Comprehensive and Very Clear Response.\n"
                "Offers a detailed description of personal experience using transfer learning in TensorFlow.\n"
                "Provides specific examples of projects where transfer learning was applied.\n"
                "Demonstrates strong understanding of transfer learning concepts and their practical application.\n"
                "Explains how transfer learning benefited those projects.\n"
                "Response is very clear, well-organized, and reflects deep engagement with the subject."
            ),
            3: (
                "Specific Experience with Basic Explanation.\n"
                "Describes personal experience with transfer learning in TensorFlow.\n"
                "Provides examples of projects where transfer learning was applied.\n"
                "Explains how transfer learning benefited those projects.\n"
                "Explanation may be brief or not fully comprehensive insight."
            ),
            2: (
                "General Response with Limited Details.\n"
                "Mentions transfer learning or TensorFlow in general terms.\n"
                "Provides minimal details about personal experience.\n"
                "Does not clearly explain how transfer learning benefited the projects.\n"
                "Shows basic understanding but lucks depth and specificity."
            ),
            1: "Minimal or Vague Response.",
            0: "Unanswered"
        },
        "keywords": {
            4: ["transfer learning","mobilenet","vgg","efficient","pretrained"],
            3: ["transfer learning"],
            2: ["tensorflow"],
            1: []
        }
    },

    3: {
        "levels": {
            4: (
                "Comprehensive and Very Clear Response.\n"
                "Offers a detailed description of a complex TensorFlow model built.\n"
                "Provides specific details about the model’s architecture, features, and purpose.\n"
                "Clearly explains the steps taken to ensure both accuracy and efficiency, such as preprocessing, model optimization techniques, regularization methods, and performance tuning.\n"
                "Demonstrates strong understanding of TensorFlow and machine learning concepts.\n"
                "Response is very clear, well-organized, and reflects deep engagement with the subject."
            ),
            3: (
                "Specific Model with Basic Explanation.\n"
                "Describes a complex TensorFlow model they have built.\n"
                "Provides some details about the model’s architecture or purpose.\n"
                "Explains steps taken to ensure accuracy and efficiency, but may lack depth.\n"
                "Explaination may be brief or not fully comprehensive."
            ),
            2: (
                "General Response with Limited Details.\n"
                "Mentions building a TensorFlow model in general terms.\n"
                "Provides minimal details about the model's complexity.\n"
                "Does not clearly explain the steps taken to ensure accuracy and efficiency.\n"
                "Shows basic understanding but lucks depth and specificity."
            ),
            1: "Minimal or Vague Response.",
            0: "Unanswered"
        },
        "keywords": {
            4: ["accuracy","optimization","regularization","preprocessing","tuning","callback","early stopping"],
            3: ["model","accuracy","optimization"],
            2: [],
            1: []
        }
    },

    4: {
        "levels": {
            4: (
                "Comprehensive and Very Clear Response.\n"
                "Provides a detailed explanation of how to implement dropout in a TensorFlow model, including code examples or specific functions (e.g., using tf.keras.layers.Dropout).\n"
                "Clearly explains the effect of dropout on training, such as how it helps prevent overfitting by randomly deactivating neurons during training.\n"
                "Discusses the impact on model performance, generalization, and possibly mentions considerations like dropout rates.\n"
                "Demonstrates strong understanding of TensorFlow and machine learning concepts.\n"
                "Response is very clear, well-organized, and reflects deep engagement with the subject."
            ),
            3: (
                "Specific Explanation with Basic Understanding.\n"
                "Explains how to implement dropout in a TensorFlow model with some specifics (e.g., mentions using Dropout layer).\n"
                "Describes the general effect of dropout on training, such as preventing overfitting.\n"
                "May omit some details about implementation or effects.\n"
                "Demonstrates a reasonable understanding but lacks comprehensive insight."
            ),
            2: "General Response with Limited Details.",
            1: "Minimal or Vague Response.",
            0: "Unanswered"
        },
        "keywords": {
            4: ["dropout", "rate", "overfitting", "random", "neuron", "impact","deactivating", "turning", "off"],
            3: ["dropout","prevent overfitting"],
            2: ["dropout"],
            1: []
        }
    },

    5: {
        "levels": {
            4: (
                "Comprehensive and Very Clear Response.\n"
                "Provides a detailed, step-by-step description of building a CNN in TensorFlow for image classification.\n"
                "Covers all key components, including data loading and preprocessing, defining the CNN layers (convolutional, pooling, activation functions), compiling the model with appropriate loss function and optimizer, training the model, and evaluating performance.\n"
                "May include code examples or specific TensorFlow functions used.\n"
                "Demonstrates strong understanding of CNNs and TensorFlow.\n"
                "Response is very clear, well-organized, and reflects deep engagement with the subject."
            ),
            3: (
                "Specific Explanation with Basic Understanding.\n"
                "Describes the process of building a CNN in TensorFlow with some specifics.\n"
                "Includes key steps such as data preprocessing, defining the CNN architecture, compiling the model, and training.\n"
                "May lack comprehensive detail or omit some important aspects.\n"
                "Demonstrates reasonable understanding but may not fully elaborate."
            ),
            2: "General Response with Limited Details.",
            1: "Minimal or Vague Response.",
            0: "Unanswered"
        },
        "keywords": {
            4: ["cnn","convolutional","pooling","activation","compile","optimizer","training"],
            3: ["cnn","architecture"],
            2: [],
            1: []
        }
    }
}

# ---- RUBRIC EVALUATOR ----
def score_text_for_question(qid: int, text: str):
    norm = normalize_text(text)
    info = RUBRIC.get(qid)

    # Kosong → nilai 0
    if not norm or len(norm) < 5:
        return 0, pick_one_reason(info["levels"][0]), []

    # LEVEL 4 → minimal 4 keyword
    kw4 = info["keywords"].get(4, [])
    hits4 = [kw for kw in kw4 if kw in norm]
    if len(hits4) >=4:
        return 4, pick_one_reason(info["levels"][4]), hits4

    # LEVEL 3 → minimal 1 keyword
    kw3 = info["keywords"].get(3, [])
    hits3 = [kw for kw in kw3 if kw in norm]
    if hits3:
        return 3, pick_one_reason(info["levels"][3]), hits3

    # LEVEL 2 → minimal 1 keyword
    kw2 = info["keywords"].get(2, [])
    hits2 = [kw for kw in kw2 if kw in norm]
    if hits2:
        return 2, pick_one_reason(info["levels"][2]), hits2

    # FALLBACK
    wc = len(norm.split())
    if wc >= 20:
        return 2, pick_one_reason(info["levels"][2]), []
    if wc >= 5:
        return 1, pick_one_reason(info["levels"][1]), []

    return 0, pick_one_reason(info["levels"][0]), []

# ---- MAIN INTERVIEW LOOP ----
results = []
interview_total = 0

for idx, wav in enumerate(wav_paths, start=1):
    print(f"\n--- Processing file {idx}: {wav} ---")

    res = model.transcribe(wav, language="en", verbose=False, condition_on_previous_text=False)

    # remove duplicate segments
    cleaned_segments = []
    prev = ""
    for seg in res["segments"]:
        segt = seg["text"].strip()
        if segt != prev:
            cleaned_segments.append(segt)
        prev = segt

    transcript = " ".join(" ".join(cleaned_segments).split())

    # confidence
    conf = overall_confidence_from_segments(res["segments"])

    # WER
    base = Path(wav).stem
    ref = references.get(base, "")
    cur_wer = wer(ref, transcript) if ref else 0.0
    cur_acc = round((1 - cur_wer) * 100, 2)

    # RUBRIC scoring
    score, reason, evidence = score_text_for_question(idx, transcript)
    interview_total += score

    results.append({
        "id": idx,
        "question": QUESTIONS[idx],
        "transcription": transcript,
        "confidence": round(conf, 4),
        "wer": cur_wer,
        "accuracy": cur_acc,
        "score": score,
        "reason": reason,
        "evidence": evidence
    })

    print("Transcript:", transcript)
    print("Score:", score)
    print("Reason:", reason)
    print("Evidence:", evidence)
    print("Confidence:", conf)
    print("WER:", cur_wer)
    print("Accuracy:", cur_acc)



--- Processing file 1: capstone_output/audios/interview_question_1.wav ---


100%|██████████| 9300/9300 [00:09<00:00, 1028.80frames/s]


Transcript: share any specific challenges you faced while working on certification and how you overcome them ah, ok actually for the challenges, there are some challenges when I took the certification especially for the project submission that I already working with it the first one is actually to meet the specific accuracy or violation laws for the evaluation matrix and yeah actually that's just need to take some trial and error with the different architecture for example like we can try to add more layer, more neurons, changes the neurons or even I also apply the dropout layer so it really helps with the validation loss to become more lower and I think that's one of the biggest challenges that I have while working on this certification
Score: 4
Reason: Response is very clear, well-organized, and reflects the learning process.
Evidence: ['challenge', 'overcome', 'error', 'architecture', 'certification']
Confidence: 0.6701160596140153
WER: 0.007936507936507936
Accuracy: 99.21

--- Proc

100%|██████████| 11472/11472 [00:10<00:00, 1066.05frames/s]


Transcript: Can you describe your experience with transfer learning and time short flow? How do you benefit from the projects? About transfer learning is actually we use existing train model from time short flow for example like VGC16, VGC19 especially for some cases that we need to use deep learning using Keras applications for example like image classification we can use transfer learning models which is that's already train model with exceptionally high accuracy, high performance, even though it's trained with different datasets but it really helps to improve our model performance, model accuracy, model loss and you know for example like mobile net, VGG19, VGG16, efficient net, it will help to improve our our models comparing to the one if you use a traditional CNN model yeah CNN model with the convolutional 2d yeah max pooling and yeah it it's it's quite good actually to use transfer learning it really helps with our model performance to improve our model performance
Score: 3
Reaso

...

Transcript: wait what is this? describe a complex TensorFlow model you have built and the steps you took to ensure its accuracy and efficiency hmmm, a complex TensorFlow model you have built and steps you took to ensure its accuracy okay I will take one of my previous project that I use yeah I also use Keras or TensorFlow model it's it is about a celiac disease prediction yeah yeah this is also I use the research project for my undergraduate thesis yeah for my script C and I use this model it's quite challenging even though it's achieved high accuracy with some dense layer, with some drawout layer and trial and error also with the callback function with the neurons but the problem is the dataset is not balanced so it has the imbalanced class datasets and the approach that I use is to just to use the technique called smooth and synthetic oversampling technique with edited nearest neighbor which is basically it's just oversampling and undersampling the datasets it helps with the accuracy

100%|██████████| 12000/12000 [00:11<00:00, 1008.13frames/s]


Transcript: explain how you implement dropout in test on flow model and test on training previously i also have implement the dropout layer also in the project submission within this certifications and we can just add the dropout layer for example if I'm not mistaken it's I have used this dropout layer in the the one that the case is image classifications yeah a German traffic something if I'm not wrong I have used this dropout layer in in in the not in the last in the middle of the layer so there's a flattened layer right not flatten the convolutional layer and then the flattened layer and I use that dropout layer which is I use with the rate of 0.2 or 0.5 if I'm not wrong and then the dense layer and the last the output layer right the effect is it will really helps to improve our accuracy and lower our validation loss by turning off some of the previous layer, for example like we have dense layer 64 and the next layer we implement the dropout layer with the rate of 0.5 and it will t

100%|██████████| 12000/12000 [00:12<00:00, 950.02frames/s]

Transcript: Describe the process of building or configuration of your image application okay, the CNN one, right? so, at the first time, of course, we need to make sure there are split, the image folder is split for each class and then we can use Keras preprocessing image dataset from directory to split the training and the validation dataset of course we can use another set which is the test dataset but yeah okay the next one we can we can just maybe we we need to implement also the image augmentation yeah data data image data augmentation to to make our data set more veritive right for example like we can rotate we can zoom it we can crop it yeah and yeah the last thing yeah of course we we can build our chain model with the convolutional 2D specify the filters the kernel size the LR activation of course the input shape for the first layer and then we can apply the max pooling 2D yeah and the next layer we can just use use conflux 2d, mix pooling and whatever it is and after that we 

In [ ]:
final_payload = {
    "assessorProfile": {
        "id": 47,
        "name": "AutoSystem",
        "photoURL": "auto.png"
    },
    "decision": "Need Human" if interview_total < 16 else "PASSED",
    "scoresOverview": {
        "project": 100,
        "interview": interview_total,
        "total": 94.3
    },
    "reviewChecklistResult": {
        "interview": {
            "minScore": 0,
            "maxScore": 4,
            "scores": results
        }
    },
    "overallNotes": "Interview responses appear consistent."
}

out_path = os.path.join(OUT_DIR, "FINAL_PAYLOAD.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(final_payload, f, indent=2)

print("Saved:", out_path)
print("Interview total score:", interview_total)

Saved: capstone_output/FINAL_PAYLOAD.json
Interview total score: 18
